# Delete report with year2 (overlapping)

In [48]:
import pandas as pd

In [49]:
report_all = pd.read_csv("/opt/hdd_1/research_hub/csr_project/CSR Reporting/Dataset/report_all.csv")

In [50]:
report_all_without_overlap = report_all[report_all['year2'].isna()]
report_all_without_overlap.drop(columns=['year2'], inplace=True)
report_all_without_overlap.to_csv("/opt/hdd_1/research_hub/csr_project/CSR Reporting/Dataset/report_all_without_overlap.csv", index=False)

/tmp/ipykernel_1028577/2918103659.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_all_without_overlap.drop(columns=['year2'], inplace=True)


# Filing csr report

In [2]:
import pandas as pd